# Исследование надежности заемщиков.

## Цель проекта
Обосновать идею кредитного отдела банка по использованию кредитного скоринга для принятия решения о кредитоспособности нового клиента на основе данных о его семейном статусе по двум параметрам: семейное положение и количество детей.  Кредитоспособность предлагается измерять одним параметром: факт погашения кредита в срок. 

## Задача проекта
на основе анализа собираемой банком статистики о платёжеспособности клиентов провести исследование по запросу кредитного отдела банка на тему **"влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок"** 

## Предварительные соображения

### Уточнение формулировки задачи

Выше в кавычках дана оригинальная формулировка от кредитного отдела.  Однако можно предположить, что собственно направление причинно-следственной связи между семейным статусом и кредитоспособностью не является предметом исследования.  Тогда задача упрощается - нужно лишь исследовать корреляцию между статистикой данных о семейном положении и детях по отношению к данным о непогашении кредита в срок.

### Первые идеи по направлению поиска решения

На первый взгляд нужно сделать следующее (после предобработки данных):
* взять выборку группы неотдавших кредит в срок
* взять *случайную* выборку того же количества из всех кредитных клиентов как контрольную группу
* оценить по количеству клиентов в выборках статистическую погрешность метода для данного набора данных (порядка `1/SQRT(n)`)
* посчитать корреляторы для обеих групп между семейным статусом (положение + кол-во детей) и кредитоспособностью (факт возврата кредита в срок)
* сравнить разницу значений корреляторов для целевой и репрезентативной выборки - сопоставить с оценкой погрешности метода 

### Дополнительные соображения

1. коррелятор, возможно, будет матрицей размерности `n` на `m`, где `n` и `m` - рассматриваемое в анализе количество значений параметров семейного положения и кол-ва детей, соответственно.  При этом, для количества детей возможно ограничить множество значений категоризацией типа `3+ детей = МНОГО(детный)`.
2. строго говоря, направление причинно-следственно связи все же важно для цели проекта (так как именно в данном направлении - от семейного статуса к кредитоспособности - принимается решение кредитным отделом для новых клиентов), но брать целевые группы в обратном порядке - выбирать из семейных с разным кол-вом детей - такая задача гораздо объемнее обратной логики выбора целевой группы из непогасивших кредит в срок - одинаковой для всех вариантов семейного статуса.  Поэтому для первой итерации модели пока принимаем утверждение выше в разделе *Уточнение формулировки задачи*.

## Предобработка данных

### Шаг 1. Обзор данных

Бросим первый взгляд на данные статистики кредитоспособности, которые придется анализировать для решения задачи

In [1]:
import pandas as pd

stat_df = pd.read_csv('/datasets/data.csv')

stat_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Сопоставим с предоставленным нам описанием семантики данных:
    •	`children` — количество детей в семье
    •	`days_employed` — общий трудовой стаж в днях
    •	`dob_years` — возраст клиента в годах
    •	`education` — уровень образования клиента
    •	`education_id` — идентификатор уровня образования
    •	`family_status` — семейное положение
    •	`family_status_id` — идентификатор семейного положения
    •	`gender` — пол клиента
    •	`income_type` — тип занятости
    •	`debt` — имел ли задолженность по возврату кредитов
    •	`total_income` — ежемесячный доход
    •	`purpose` — цель получения кредита

#### Первые выводы:
1. отсутствует ID самого клиента, и нет флага повторного обращения за кредитом, так что запись соответствует инстансу процесса предоставления кредита, но несколько записей могут относится и к одному тому же клиенту.  Хотя напрямую это не мешает решению задачи, но следует учитывать при трактовке записей в таблице; 
2. именования колонок - все ОК, возможны невидимые пока конечные пробелы и киррилические символы, но вероятность мала;
3. типы данных в колонках уже приведены к обрабатываемым математическими функциями ОК, однако многие удобнее превратить в категорийные - например, задолженность в `bool`.  Также замечание: почему стаж в днях не целое, а с плавающей точкой?
4. судя по кол-ву `non-null`, данные весьма полны (качество пока не известно), неполные данные видны только в колонках:
    * трудовой стаж в днях - но данный параметр вообще вызывает сомнения в достоверности, так как его точно знает только пенсионный фонд.  Не каждый знает свой стаж даже с точностью +/- год!
    * ежемесячный доход - скорее всего, нет данных в сегменте больших доходов, или они недостоверны (условная цифра)
   но оба параметра не относятся к списку исследуемых напрямую, поэтому их неполнота вряд ли создаст проблемы решению задачи проекта;
5. общее количество данных (21.5K записей) позволяет надеятся на уровень статистической погрешности менее `1%`.  Но проверим сколько в этих данных данных о невозврате кредитов:

In [2]:
print(stat_df['debt'].value_counts()) #на всякий случай убедимся что там только 0 и 1 - а то ниже используем sum() 
print('_________________________________________________________________\n')
print(f'Всего в данных статистики присутствует {stat_df["debt"].sum()} должников')
print(f'Порядок ожидаемой статистической ошибки в выборке должников: {round((stat_df["debt"].sum())**(-1/2)*100, 1)} %')

0    19784
1     1741
Name: debt, dtype: int64
_________________________________________________________________

Всего в данных статистики присутствует 1741 должников
Порядок ожидаемой статистической ошибки в выборке должников: 2.4 %


**Вывод: предоставленные данные в целом соответствуют поставленной задаче, для достоверности вывода исследования ожидаемая корреляция между семейным статусом и кредитоспособностью должна превысить уровень 2-3 статистических ошибок, то есть быть на уровне 5-7%**

Посмотрим более детально на данные в каждом столбце, однако предварительно проведем небольшое исследование (в роли гостьи из будущего), необходимость в результатах которого возникнет позже в разделе 4.16 - поисследуем вопрос отсутствующих данных:

Нас беспокоит сколько записей с одновременным отсутствием данных в разных колонках таблицы:

In [3]:
missing_douplets = stat_df.loc[(stat_df['days_employed'].isna()) & (stat_df['total_income'].isna()), 'children'].count()
print(f'записей с одновременно отсутствующей информацией в колонках стажа работы и ежемесячного дохода = {missing_douplets}')

print('всего записей в таблице = 21525')
print('кол-во записей в колонках стажа работы и ежемесячного дохода одинаковое = 19351')
print(f'заметим, что 21525 - 19351 = {21525 - 19351}')

записей с одновременно отсутствующей информацией в колонках стажа работы и ежемесячного дохода = 2174
всего записей в таблице = 21525
кол-во записей в колонках стажа работы и ежемесячного дохода одинаковое = 19351
заметим, что 21525 - 19351 = 2174


Проверим также искомую в пункте 4.16 корреляцию между дупликатами и отсутствием информации в колонках стажа работы и ежемесячного дохода: 

In [4]:
print(f'Всего дупликатов {stat_df.duplicated().sum()}')
print('__________________________________\n')
stat_df.loc[stat_df.duplicated(),['days_employed','total_income']]

Всего дупликатов 54
__________________________________



,days_employed,total_income
2849,NaN,NaN
4182,NaN,NaN
4851,NaN,NaN
5557,NaN,NaN
7808,NaN,NaN
8583,NaN,NaN
9238,NaN,NaN
9528,NaN,NaN
9627,NaN,NaN
10462,NaN,NaN


Итак, обнаружены две особенности в отношении отсутствующих данных: 
1. **информация о стаже работы и ежемесячном доходе отсутствует в записях строго одновременно**
2. **все дупликаты попадают на строки с отсутствующей информацией**

мы вернемся к этому наблюдению в главе 4.16 при обработке полных дупликатов

#### Начнем с предварительного ознакомления с данными каждого столбца

In [5]:
for col in stat_df.columns:
    print(f'_________________ {col} ___________________\n')
    print(stat_df[col].value_counts())
    print('___________________________________________ \n')

_________________ children ___________________

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
___________________________________________ 

_________________ days_employed ___________________

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64
___________________________________________ 

_________________ dob_years ___________________

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    35

**Режущих глаз замечаний так много, что оптимальнее выполнять программу шагов предобработки данных для каждого столбца отдельно** Только обработку дупликатов оставим напоследок - так как уже видно, что после унификации значений в таких колонках, как `purpose` их кол-во может увеличится. 

### Шаг 2.1 Заполнение пропусков
выполним ниже для каждого столбца отдельно

### Шаг 2.2 Проверка данных на аномалии и исправления
выполним ниже для каждого столбца отдельно

### Шаг 2.3. Изменение типов данных
выполним ниже для каждого столбца отдельно

### Столбец `children` (количество детей в семье):
   * наблюдается (см. здесь и далее вывод кода в пункте 4.1.3 выше) **отрицательное или нереально большое кол-во** в нескольких позициях, 
   
     от общей статистики их порядка 0.57%, посмотрим сколько их от выборки по неплательщикам

In [6]:
print(f'Кол-во должников с аномальным кол-вом детей: \
      {stat_df[(stat_df["debt"] == 1) & ((stat_df["children"] == -1) | (stat_df["children"] == 20))]["children"].count()}')
print(f'относительно общего кол-ва должников это составит порядка {round(9/1741*100,2)} %')

Кол-во должников с аномальным кол-вом детей:       9
относительно общего кол-ва должников это составит порядка 0.52 %


Существенной корреляции между браком данных по кол-ву детей и неплатежеспособностью не видно, поэтому можно либо раскидать записи с бракованными данными случайным образом пропорционально весам категорий по кол-ву детей от 0 до 5, либо просто их выкинуть из таблицы.  Замена на медианное значение не лучшая идея - так как это категорийный параметр и мы увеличим баласт одной из категорий!

Принятое решение: ниже опишем подход к раскидыванию случайным образом пропорционально весам категорий, а на практике просто выкинем - разница в уменьшении статистического размера выборок на полпроцента!

* сперва определяем веса вариантов 0,1,2,3,4,5 детей в подмножестве строк с небракованными данными
* затем помещаем (то есть устанавливаем значение параметра `children`) в категорию "0" для N-строк (случайно выбранных из бракованных 76+47=123) так чтобы N = round(123\*вес варианта "0"
* продолжаем для категорий 1,2,3,4, а последнюю 5 помещаем оставшиеся из 123 бракованных строк
* вопрос случайного выбора строк пока сложен - пока нет владения соответствующими средствами Python и Pandas (поэтому выбран статистически эквивалентный вариант выброса этих 123 строк)

In [7]:
stat_df = stat_df[(stat_df['children'] != -1) & (stat_df['children'] != 20)]
stat_df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Столбец `days_employed` (общий трудовой стаж в днях):
   * невразумительные значения во всех позициях,
    
   можно предположить, что какая-то функция автозаполняет эту позицию, например, временным интервалом до/от даты первого найма на работу.  Нужно еще "повертеть" данный столбец, чтобы проверить данную гипотезу:

In [8]:
print(f'Диапазон значений от мин {stat_df["days_employed"].min()} до макс {stat_df["days_employed"].max()}')

Диапазон значений от мин -18388.949900568383 до макс 401755.40047533


верхняя граница диапазона выглядит еще хуже нижней: 400000 дней - это более 1000 лет!
Посмотрим, много ли таких "долгожителей-стахановцев" - посчитаем сколько записей с абсолютным значением дней более максимального стажа в 50 лет:

In [9]:
print(f'Менее 1 года стажа у {stat_df[ abs(stat_df["days_employed"]) < 1 * 365]["days_employed"].count()}')
print(f'Менее 5 лет стажа у {stat_df[ abs(stat_df["days_employed"]) < 5 * 365]["days_employed"].count()}')
print(f'Менее 10 лет стажа у {stat_df[ abs(stat_df["days_employed"]) < 10 * 365]["days_employed"].count()}')
print(f'Менее 20 лет стажа у {stat_df[ abs(stat_df["days_employed"]) < 20 * 365]["days_employed"].count()}')
print(f'_________________________ {round(15149/19351*100,1)} % выше этой линии __________________________')
print(f'Более 20 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 20 * 365]["days_employed"].count()}')
print(f'Более 30 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 30 * 365]["days_employed"].count()}')
print(f'Более 40 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 40 * 365]["days_employed"].count()}')
print(f'Более 50 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 50 * 365]["days_employed"].count()}')
print(f'_________________________ {round(3445/19351*100,1)} % ниже этой линии __________________________')
print(f'Более 100 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 100 * 365]["days_employed"].count()}')
print(f'Более 500 лет стажа у {stat_df[ abs(stat_df["days_employed"]) > 500 * 365]["days_employed"].count()}')

Менее 1 года стажа у 1817
Менее 5 лет стажа у 8582
Менее 10 лет стажа у 12676
Менее 20 лет стажа у 15057
_________________________ 78.3 % выше этой линии __________________________
Более 20 лет стажа у 4183
Более 30 лет стажа у 3608
Более 40 лет стажа у 3448
Более 50 лет стажа у 3432
_________________________ 17.8 % ниже этой линии __________________________
Более 100 лет стажа у 3431
Более 500 лет стажа у 3431


Если придерживаться предположения, что абсолютное целое от `days_employed` соответствует кол-ву дней с даты найма на работу до
* либо сегодняшнего дня (могло бы объяснить рост со временем хвоста распределения, но не на 100-500 лет!);
* либо дня запроса кредита (хуже, так как не объясняет даже рост хвоста распределения),


то это соответствует колоколообразному растределению возле нуля для 82% записей с дисперсией 5 лет при 18% записей в нереально длинном хвосте до 1000 лет!  Чушь - 18% не выкинуть как флуктуацию!

Единственное разумное объяснение - ошибка приведения типов данных.  Наверное, где-то в исходниках это была календарная дата или строка с календарной датой, но в какой-то момент произошло чтение как типа float из той же ячейки памяти, которое необратимо хаотизировало информацию.  Необходим новый запрос к источнику на исправление файла данных.  Поскольку в условиях проекта мы продолжаем до конца, то оставим данный данный столбец как есть - к тому же, поскольку, хоть семантика информации и потеряна, но в сравнении полныех дупликатов он еще может участвовать!

P.S.: Есть еще совсем безумная идея, откуда мог появиться хвост - это могут быть кредиты, выданные до 2000 года, когда год писался в системах двумя последними цифрами.  Система могла прочитать даты в данных о них от Рождества Христова и добавить 1900 лет в стаж заемщикам, но в этом случае хвост растянулся бы на 1900 лет или 690 000 дней - а у нас данные обрываются на 402 тысяче дней, так что идея красивая, но числа не сходятся.

### Столбец `dob_years` (возраст клиента в годах):

выглядит неплохо - распределение от 19 до 75 (оба края редко втречаются), самые частые значения в 35-45 лет.  одно замечание - 101 запись со значением возраста 0 (при этом ни одной записи от 1 до 18 лет).  То есть 0 является плейс-холдером и равнозначен отсутствию данных.

От общего количества записей это составит 0,47 %, посмотрим есть ли корреляция с неплательщиками:

In [10]:
print(f'Кол-во должников с отсутствием данных о возрасте: \
      {stat_df[(stat_df["debt"] == 1) & (stat_df["dob_years"] == 0)]["dob_years"].count()}')
print(f'относительно общего кол-ва должников это составит порядка {round(8/1741*100,2)} %')

Кол-во должников с отсутствием данных о возрасте:       8
относительно общего кол-ва должников это составит порядка 0.46 %


Как и в случае со столбцом `children` корреляции нет, поэтому можно либо случайно раскидать по категориям возрастов (после категоризации) сохранив веса в распределени остальных записей по категориям, либо выкинуть - учитывая, что это ухудшит статистическую мощность данных лишь на полпроцента от их количества.  Что мы и сделаем:

In [11]:
stat_df = stat_df[stat_df['dob_years'] != 0]
stat_df.info() # то ли мы отрезали

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21302 non-null  int64  
 1   days_employed     19150 non-null  float64
 2   dob_years         21302 non-null  int64  
 3   education         21302 non-null  object 
 4   education_id      21302 non-null  int64  
 5   family_status     21302 non-null  object 
 6   family_status_id  21302 non-null  int64  
 7   gender            21302 non-null  object 
 8   income_type       21302 non-null  object 
 9   debt              21302 non-null  int64  
 10  total_income      19150 non-null  float64
 11  purpose           21302 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Столбцы `education` (уровень образования клиента) и `education_id` (идентификатор уровня образования):

* первое, что бросается в глаза, - уникальных значений в `education` гораздо больше, чем в `education_id`, что не имеет смысла.  Поэтому сперва устраним разное описание для одинакового образования - судя по внешнему виду, разница в регистре, поэтому достаточно привети все строки к прописному регистру:


In [12]:
stat_df['education'] = stat_df['education'].str.lower()

print(stat_df['education'].value_counts())
print('___________________________________________________\n')
print(stat_df['education_id'].value_counts())

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64
___________________________________________________

1    15073
0     5202
2      739
3      282
4        6
Name: education_id, dtype: int64


теперь соответствует, хотя порядок нумерации уровней образования в ID вызывает удивление!

Чтобы избежать дупликации информации в таблице создаем отдельный словарь для расшифровки `education_id`, а столбец `education` удаляем:

In [13]:
data = [
    [0, 'высшее'],
    [1, 'среднее'],
    [2, 'неоконченное высшее'], 
    [3, 'начальное'],
    [4, 'ученая степень']
]
columns = ['education_id', 'education']

education_dict = pd.DataFrame(data=data, columns=columns)

print(education_dict)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


In [14]:
stat_df = stat_df.drop(columns='education')
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21302 non-null  int64  
 1   days_employed     19150 non-null  float64
 2   dob_years         21302 non-null  int64  
 3   education_id      21302 non-null  int64  
 4   family_status     21302 non-null  object 
 5   family_status_id  21302 non-null  int64  
 6   gender            21302 non-null  object 
 7   income_type       21302 non-null  object 
 8   debt              21302 non-null  int64  
 9   total_income      19150 non-null  float64
 10  purpose           21302 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 2.0+ MB


### Столбцы `family_status` (семейное положение) и `family_status_id` (идентификатор семейного положения):

здесь количество уникальных значений между двумя столбцами полностью соответствует - и по их частоте встречи в таблице, поэтому сразу выделяем словарь и удаляем дублирующий столбец.  Единственный момент - для единообразия опустим регистр букв при создании словаря в нижний:

In [15]:
data = [
    [0, 'женат / замужем'],
    [1, 'гражданский брак'],
    [2, 'вдовец / вдова'], 
    [3, 'в разводе'],
    [4, 'не женат / не замужем']
]
columns = ['family_status_id', 'family_status']

family_status_dict = pd.DataFrame(data=data, columns=columns)

print(family_status_dict)

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


In [16]:
stat_df = stat_df.drop(columns='family_status')
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21302 non-null  int64  
 1   days_employed     19150 non-null  float64
 2   dob_years         21302 non-null  int64  
 3   education_id      21302 non-null  int64  
 4   family_status_id  21302 non-null  int64  
 5   gender            21302 non-null  object 
 6   income_type       21302 non-null  object 
 7   debt              21302 non-null  int64  
 8   total_income      19150 non-null  float64
 9   purpose           21302 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 1.8+ MB


### Столбец `gender` (пол клиента):
все в порядке - только надо удалить одну запись с аномальным значением:

In [17]:
stat_df = stat_df[stat_df['gender'] != 'XNA']
stat_df['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

### Столбец `income_type` (тип занятости):
этот столбец не мешает и не помогает.  К тому же не понятен принцип категоризации на сотрудников, компаньонов и предпринимателей, да и факт, что последних только два, только усиливает интригу - я думал, что кредиты берут на бизнес, видимо, данная кредитная организация работает в домашнем секторе кредитования.

Решение - оставим как есть.


### Столбец `debt` (имел ли задолженность по возврату кредитов):
по семантике должен быть логический тип True/False, но можно оставить и целый тип, коль скоро проверено, что все значения только 0 и 1.  Других претензий нет.

### Столбец `total_income` (ежемесячный доход) - начало предобработки:
важный для кредитного решения столбец, хотя обычно принимается во внимание соотношение месячной выплаты по кредиту и месячного дохода, но таких данных нет, поэтому будем мешать яблоки с апельсинами.  Частично роль уровня месячной выплаты может сыграть цель кредита (далее).  Проведем категоризацию записей по уровню дохода.  Предварительно оценим параметры распределения:

In [18]:
print(f'диапазон дохода от {stat_df["total_income"].min()} до {stat_df["total_income"].max()}')
print(f'средний доход (задекларированный) {stat_df["total_income"].mean()}')
print(f'медиана распределения на уровне дохода {stat_df["total_income"].median()}')

диапазон дохода от 20667.26379327158 до 2265604.028722744
средний доход (задекларированный) 167491.8688048186
медиана распределения на уровне дохода 145017.93753253992


In [19]:
total_lines = stat_df['total_income'].size
nool_lines = stat_df['total_income'].isna().sum()
print(f'При этом в оставшейся таблице отсутствует информация о доходах для \
{round(nool_lines / total_lines*100,1)} % всех записей')


При этом в оставшейся таблице отсутствует информация о доходах для 10.1 % всех записей


Предполагая, что информация о доходах скрывается в сегменте больших кредитов, выкидывать эту статистику нельзя, так как точность кредитного скоринга желательна именно в области больших кредитов.  Идей чем заполнять немного, но усложним немного способ, приняв во внимание упомянутую выше взаимосвязь между целами кредита (как косвенное представление типичных суммы и срока кредита) - будем заполнять медианным значением для группы записей с той же целью кредита!

Поэтому пока переключимся на предварительную обработку столбца с целью кредита, потом удалим полные дупликаты в записях, так как они сильно могут подвинуть медиану, и вернемся к заполнению пропущенных значений месячного дохода - до его категоризации. 

### Столбец `purpose` (цель получения кредита):
сам столбец оставим как есть, но добавим столбец категории в таблицу.  Для этого сперва определим функцию принимающую значения столбца `purpose` и возврещающую категорию из требуемого списка значений:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'


In [20]:
def categorize_purpose(s):
    if 'автомобил' in s: return 'операции с автомобилем'
    elif ('недвижим' in s) or ('жиль' in s): return 'операции с недвижимостью'
    elif 'свадьб' in s: return 'проведение свадьбы'
    elif 'образован' in s: return 'получение образования'
    else: return "категория цели не определена"

stat_df['purpose_category'] = stat_df['purpose'].apply(categorize_purpose)

stat_df['purpose_category'].value_counts()

операции с недвижимостью    10732
операции с автомобилем       4267
получение образования        3979
проведение свадьбы           2323
Name: purpose_category, dtype: int64

Категоризация прошла успешно, возвращаемся к предобработке столбца с месячным доходом.

### Столбец `total_income` (ежемесячный доход) - завершение предобработки:

#### Заполнение пробелов
следуем идее заполнения пробелов медианным значением дохода по своей категории цели кредита.  Сперва посчитаем медианные значения для всех категирий и оформими их как Python-словарь {категория цели : медианный доход внутри данной категории цели}:

In [21]:
income_dict = {}

for cat in ['операции с автомобилем','операции с недвижимостью','проведение свадьбы','получение образования']:
    temp_df = stat_df[stat_df['purpose_category'] == cat]
    income_dict[cat] = temp_df['total_income'].median()
    
print(income_dict)

{'операции с автомобилем': 143920.8437993451, 'операции с недвижимостью': 146676.7370012449, 'проведение свадьбы': 144171.24479171788, 'получение образования': 143289.35251638902}


Игра не стоила свеч!  Все медианные значения близки!  А значит, как и предполагалось, главная информация в `total_income` отсутствует - сегмент больших сумм!  Ну завершим тем не менее выбранный подход и заполним пробелы:  

In [22]:
def new_income(row):
    income = row['total_income']
    purpose_cat = row['purpose_category']

    if income == 0.0: return income_dict[purpose_cat]
    else: return income

    
stat_df.loc[stat_df['total_income'].isna(),'total_income'] = 0.0 # что-то с NaN не хочет проверяться выше - говорит неопределена
        
stat_df['total_income'] = stat_df.apply(new_income, axis=1)

stat_df['total_income'].value_counts()

146676.737001    1068
143289.352516     423
143920.843799     417
144171.244792     247
490067.335319       1
                 ... 
152410.919231       1
98360.185970        1
208520.564033       1
117329.195869       1
189255.286637       1
Name: total_income, Length: 19150, dtype: int64

#### Категоризация
Пропуски заполнены, осталось категоризовать диапазоны значений, но сперва удовлетворим дополнительному запросу поменять тип `total_income` на целое:

In [23]:
stat_df['total_income'] = stat_df['total_income'].astype('int')
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21301 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21301 non-null  int64  
 3   education_id      21301 non-null  int64  
 4   family_status_id  21301 non-null  int64  
 5   gender            21301 non-null  object 
 6   income_type       21301 non-null  object 
 7   debt              21301 non-null  int64  
 8   total_income      21301 non-null  int64  
 9   purpose           21301 non-null  object 
 10  purpose_category  21301 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 2.0+ MB


In [24]:
def categorize_income(s):
    if s < 30000 : return 'E'
    elif s < 50000 : return 'D'
    elif s < 200000 : return 'C'
    elif s < 1000000 : return 'B'
    else: return "A"

stat_df['total_income_category'] = stat_df['total_income'].apply(categorize_income)

stat_df['total_income_category'].value_counts()

C    15921
B     4986
D      347
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.4. Удаление дубликатов
осталось удалить полные дупликаты среди записей таблицы:

In [25]:
print(f'Полных дупликатов среди строк всей таблицы {stat_df.duplicated().sum()}')

Полных дупликатов среди строк всей таблицы 71


Заметим сразу, что полных дупликатов прибавилось - в начальном дата-фрейме их было 54 (см. 4.1.2), а теперь 71.  Такое увеличение произошло после раскрытия неявных дупликатов при приведении значений в колонке `education` в соответствие с `education_id` путем перевода всех вариантов в нижний регистр букв.

**Возможные причины полных дупликатов:**
   * в записях отсутствуют уникальные ID заемщиков, поэтому теоретически есть возможность полных "тезок" при разных физлицах, либо повторное взятие кредита в тот же год одним и тем же физлицом
   * или есть вероятность, что записи попадают в данную статистику разными каналами и могут дублироваться по данной причине.

На всякий случай посмотрим еще детали про полные дупликаты:

In [26]:
full_double_df = stat_df[stat_df.duplicated()] #это таблица повторных записей
full_double_double_ser = full_double_df.duplicated() #это выбор записей, которые повторяются более двух раз

print(f'Более чем два раза повторяются {full_double_double_ser.sum()} записей')
print('____________________________________________________________________\n')
print('Вот какие записи повторяются более двух раз:')
full_double_df[full_double_double_ser]

Более чем два раза повторяются 5 записей
____________________________________________________________________

Вот какие записи повторяются более двух раз:


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
5557,0,NaN,58,1,1,F,пенсионер,0,144171,сыграть свадьбу,проведение свадьбы,C
9604,0,NaN,71,1,1,F,пенсионер,0,144171,на проведение свадьбы,проведение свадьбы,C
10994,0,NaN,62,1,0,F,пенсионер,0,146676,ремонт жилью,операции с недвижимостью,C
20662,0,NaN,58,1,1,M,сотрудник,0,144171,на проведение свадьбы,проведение свадьбы,C
21132,0,NaN,47,1,0,F,сотрудник,0,146676,ремонт жилью,операции с недвижимостью,C


Заметим, что вследствие отсутствия идентификатора клиента или регистрационного номера заявки (то есть ID процесса), действительно уникальными столбцами в записях являются только `days_employed` и `total_income`.  Но именно по ним наблюдается отсутствие данных (заполнение нами `total_income` медианным значением ничего не поменяло - заполненные значения утратили уникальность). Все же остальные параметры записи принимают столь ограниченные значения, что даже их случайных комбинаций всего  6\*(75-18)\*5\*5\*2\*4\*4 = 273 600.  

Можно предположить, что все дупликаты наблюдаются только в записях с отсутствием исходной информации в колонках `days_employed` и `total_income`.  Воспользуемся "машиной времени" благодаря рестарту кернела и проверим эту гипотезу на изначальном дата-фрейме - в конце пункта 4.1.2.

Итак мы видим (после путешествия во времени в пункт 4.1.2), что:
1. множества записей с отсутствующей информаций в столбцах `days_employed` и `total_income` полностью совпали
2. все настояшие дупликаты приходятся на записи с отсутствием информации в столбцах `days_employed` и `total_income` 
2. таких записей 2174 или 10.1 % от оригинального набора данных

Посчитаем вероятность при выборке повторяемой 2174 раза из уникальных равновероятных 273 600 вариантов ни разу не выбрать дупликат (можно считать по аналитическим формулам как 273600! / ( 2174!\*(273600\*\*2174) ), но это деление очень больших чисел, у интерпретатора может быть переполнение, проще напрямую умножать вероятности не повториться 2174 раза:

In [27]:
probability = 1.0

for i in range(2175):
    probability *= (273600-i)/273600

print(f'вероятность не иметь дупликатов можно оценить как {round(probability*100,3)} %')

вероятность не иметь дупликатов можно оценить как 0.017 %


таким образом, у нас нет никаких оснований считать одинаковые записи дупликатами одного процесса - согласно нашей оценке, для такого большого количества записей с отсутствующими данными просто невероятно выбрать полностью уникальным способом набор из всех возможных значений оставшихся параметров (при отсутствии самых многозначных столбцов стажа и дохода).

**Поэтому анализ дупликатов заканчивается неожиданным волевым решением не удалять дупликаты с отсутствием исходных данных одновременно в столбцах `days_employed` и `total_income` - а таковыми у нас являются все дупликаты (см. последний параграф пункта 4.1.2). Поэтому все дупликаты признаем фальшивыми и не удаляем совсем!**

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
подзадача выполнена при предобработке соответствующих столбцов - результирующие словари-дата-фреймы вот:

In [28]:
print(education_dict)
print('__________________________________________\n')
print(family_status_dict)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
__________________________________________

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


### Шаг 2.6. Категоризация дохода.
подзадача выполнена при предобработке соответствующего столбца - добавлен столбец с категорией:

In [29]:
stat_df['total_income_category'].value_counts()

C    15921
B     4986
D      347
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.
подзадача выполнена при предобработке соответствующего столбца - добавлен столбец с категорией:

In [30]:
stat_df['purpose_category'].value_counts()

операции с недвижимостью    10732
операции с автомобилем       4267
получение образования        3979
проведение свадьбы           2323
Name: purpose_category, dtype: int64

### посмотрим на итоговый дата-фрейм после завершения предобработки:

In [31]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21301 non-null  int64  
 1   days_employed          19149 non-null  float64
 2   dob_years              21301 non-null  int64  
 3   education_id           21301 non-null  int64  
 4   family_status_id       21301 non-null  int64  
 5   gender                 21301 non-null  object 
 6   income_type            21301 non-null  object 
 7   debt                   21301 non-null  int64  
 8   total_income           21301 non-null  int64  
 9   purpose                21301 non-null  object 
 10  purpose_category       21301 non-null  object 
 11  total_income_category  21301 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


как детально анализировалось в главе 4.6, в столбце `days_employed` утрачена информация и использовать его для логического анализа не предоставляется возможным.  Поэтому удалим этот столбец для финализации предобработки.  Также перепишем индексы записей:

In [32]:
stat_df = stat_df.drop(columns='days_employed')
stat_df = stat_df.reset_index(drop=True)
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21301 non-null  int64 
 1   dob_years              21301 non-null  int64 
 2   education_id           21301 non-null  int64 
 3   family_status_id       21301 non-null  int64 
 4   gender                 21301 non-null  object
 5   income_type            21301 non-null  object
 6   debt                   21301 non-null  int64 
 7   total_income           21301 non-null  int64 
 8   purpose                21301 non-null  object
 9   purpose_category       21301 non-null  object
 10  total_income_category  21301 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


## Решение основной задачи проекта
В качестве контрольной выборки, представляющей случайный набор записей, возьмем весь дата-фрейм `stat_df`.
В качестве целевой группы возьмем все записи по факту задолженности:

In [33]:
debt_df = stat_df[stat_df['debt'] == 1]
debt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1724 entries, 14 to 21299
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               1724 non-null   int64 
 1   dob_years              1724 non-null   int64 
 2   education_id           1724 non-null   int64 
 3   family_status_id       1724 non-null   int64 
 4   gender                 1724 non-null   object
 5   income_type            1724 non-null   object
 6   debt                   1724 non-null   int64 
 7   total_income           1724 non-null   int64 
 8   purpose                1724 non-null   object
 9   purpose_category       1724 non-null   object
 10  total_income_category  1724 non-null   object
dtypes: int64(6), object(5)
memory usage: 161.6+ KB


Посмотрим и сравним распределение всех записей для контрольной и целевой групп по пивот-матрице с координатами `children` и `family_status_id`:

In [34]:
stat_pivot = stat_df.pivot_table(index='children', columns='family_status_id', values='gender', aggfunc='count')
stat_pivot = stat_pivot / 213.01
stat_pivot = stat_pivot.fillna(0.0) 
print('Данная таблица показывает процент записей в контрольной выборке для комбинации (children, family_status_id)')
print(stat_pivot)

print('________________________________________________________________________________\n')

debt_pivot = debt_df.pivot_table(index='children', columns='family_status_id', values='gender', aggfunc='count')
debt_pivot = debt_pivot / 17.24
debt_pivot = debt_pivot.fillna(0.0)
print('Данная таблица показывает процент записей в целевой группе для комбинации (children, family_status_id)')
print(debt_pivot)


Данная таблица показывает процент записей в контрольной выборке для комбинации (children, family_status_id)
family_status_id          0          1         2         3          4
children                                                             
0                 35.064081  12.853857  3.957561  3.647716  10.572274
1                 13.966480   4.657058  0.361485  1.460025   2.098493
2                  7.168678   1.605558  0.093892  0.370875   0.347402
3                  1.159570   0.262898  0.028168  0.051641   0.037557
4                  0.136144   0.037557  0.004695  0.004695   0.009389
5                  0.032862   0.009389  0.000000  0.000000   0.000000
________________________________________________________________________________

Данная таблица показывает процент записей в целевой группе для комбинации (children, family_status_id)
family_status_id          0          1         2         3          4
children                                                             
0      

добавим строку и колонку суммарных значений - под номером 100000:

In [35]:
stat_pivot.loc[100000,:] = stat_pivot.loc[0,:] + \
                      stat_pivot.loc[1,:] + \
                      stat_pivot.loc[2,:] + \
                      stat_pivot.loc[3,:] + \
                      stat_pivot.loc[4,:] + \
                      stat_pivot.loc[5,:]
stat_pivot[100000] = stat_pivot[0]+stat_pivot[1]+stat_pivot[2]+stat_pivot[3]+stat_pivot[4]
print(stat_pivot)

family_status_id     0          1         2         3          4       \
children                                                                
0                 35.064081  12.853857  3.957561  3.647716  10.572274   
1                 13.966480   4.657058  0.361485  1.460025   2.098493   
2                  7.168678   1.605558  0.093892  0.370875   0.347402   
3                  1.159570   0.262898  0.028168  0.051641   0.037557   
4                  0.136144   0.037557  0.004695  0.004695   0.009389   
5                  0.032862   0.009389  0.000000  0.000000   0.000000   
100000            57.527816  19.426318  4.445801  5.534951  13.065114   

family_status_id      100000  
children                      
0                  66.095488  
1                  22.543543  
2                   9.586404  
3                   1.539834  
4                   0.192479  
5                   0.042252  
100000            100.000000  


аналогично для целевой группы:

In [36]:
debt_pivot.loc[5,:] = 0.0
debt_pivot.loc[100000,:] = debt_pivot.loc[0,:] + \
                      debt_pivot.loc[1,:] + \
                      debt_pivot.loc[2,:] + \
                      debt_pivot.loc[3,:] + \
                      debt_pivot.loc[4,:] + \
                      debt_pivot.loc[5,:]
debt_pivot[100000] = debt_pivot[0]+debt_pivot[1]+debt_pivot[2]+debt_pivot[3]+debt_pivot[4]
print(debt_pivot)

family_status_id     0          1         2         3          4       \
children                                                                
0                 29.814385  13.167053  3.016241  3.190255  12.180974   
1                 14.153132   6.844548  0.406032  1.218097   2.958237   
2                  8.410673   1.740139  0.174014  0.406032   0.522042   
3                  0.986079   0.464037  0.000000  0.058005   0.058005   
4                  0.174014   0.000000  0.000000  0.000000   0.058005   
5                  0.000000   0.000000  0.000000  0.000000   0.000000   
100000            53.538283  22.215777  3.596288  4.872390  15.777262   

family_status_id      100000  
children                      
0                  61.368910  
1                  25.580046  
2                  11.252900  
3                   1.566125  
4                   0.232019  
5                   0.000000  
100000            100.000000  


Ну и в итоге получим динамику для распределения доли записей по координатам `children` vs `family_status_id` при переходе от случайной выборки записей к записям по задолженности:

In [37]:
delta_pivot = debt_pivot - stat_pivot
delta_pivot.loc[100000,100000] = 0.0

print(delta_pivot)
print()
print('______________________________ где __________________________________________\n')
print(family_status_dict)

family_status_id    0         1         2         3         4         100000
children                                                                    
0                -5.249696  0.313197 -0.941319 -0.457461  1.608701 -4.726579
1                 0.186652  2.187489  0.044547 -0.241928  0.859744  3.036504
2                 1.241995  0.134581  0.080122  0.035158  0.174640  1.666496
3                -0.173491  0.201139 -0.028168  0.006364  0.020448  0.026291
4                 0.037870 -0.037557 -0.004695 -0.004695  0.048615  0.039539
5                -0.032862 -0.009389  0.000000  0.000000  0.000000 -0.042252
100000           -3.989533  2.789459 -0.849513 -0.662562  2.712148  0.000000

______________________________ где __________________________________________

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4           

### Ответы на вопросы.

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

**Ответ 1:** да есть (см. колонку 100000 пивот-таблицы выше), даже в среднем по всем семейным статусам вероятность отдать кредит повышается, если детей нет совсем - эффект вдвое превосходит уровень стат погрешности.  При этом максимальный негативный эффект наблюдается у одно и двудетных клиентов, для 3,4 и 5 детей абсолютный эффект существенно меньше уровня статистической погрешности.

Надо сказать, что выборка недостаточна для раздельного анализа на уровнях 3,4 и 5 детей - из лучше было категоризовать в одну категорию многодетных, но даже в сумме для этих строк пивот-таблицы имеем пренебрежимо малый эффект.  

#### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Ответ 2:** да есть (см. строку 100000 пивот-таблицы выше), даже при усреднении по всем вариантам кол-ва детей вероятность отдать кредит выше у женатых (полторы стат ошибки) и ниже у неженатых или состоящих в гражданском браке (порядка стат ошибки каждая категория)

#### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

посмотрим вероятность возврата в разных категориях дохода:

In [38]:
total_credits = stat_df.groupby('total_income_category')['debt'].count()
print(total_credits) # посмотрим репрезентативность выборок в категориях

debt_credits = stat_df.groupby('total_income_category')['debt'].sum()
print('______________________________________________________________________\n')

credit_returned_rel = (total_credits - debt_credits) / total_credits*100
print(credit_returned_rel)

total_income_category
A       25
B     4986
C    15921
D      347
E       22
Name: debt, dtype: int64
______________________________________________________________________

total_income_category
A    92.000000
B    92.920176
C    91.545757
D    93.948127
E    90.909091
Name: debt, dtype: float64


**Ответ 3:** зависимость слабая - в тех категориях, где много статистики.  Ожидаемой монотонности нет, а по краям в категориях A и E менее 50 записей в целом.  Существенной зависимости не обнаружено - все в пределах стат-погрешности.

#### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

посмотрим вероятность возврата в разных категориях цели:

In [39]:
total_credits = stat_df.groupby('purpose_category')['debt'].count()
print(total_credits) # посмотрим репрезентативность выборок в категориях

debt_credits = stat_df.groupby('purpose_category')['debt'].sum()
print('______________________________________________________________________\n')

credit_returned_rel = (total_credits - debt_credits) / total_credits*100
print(credit_returned_rel)

purpose_category
операции с автомобилем       4267
операции с недвижимостью    10732
получение образования        3979
проведение свадьбы           2323
Name: debt, dtype: int64
______________________________________________________________________

purpose_category
операции с автомобилем      90.696039
операции с недвижимостью    92.759970
получение образования       90.726313
проведение свадьбы          92.208351
Name: debt, dtype: float64


**Ответ 4:** зависимость прослеживается, а главное, ей можно верить, так как категоризация проведена хорошо - во всех категориях много записей.  Сформулировать можно так: операции с недвижимостью увеличивают среднюю вероятность возврата кредита (91.9%) на один процент, а вероятность возврата автокредита и кредита на образование меньше средней на полпроцента.  Свадебные цели не меняют существенно вероятность возврата кредита.

### Выводы:

#### Вывод 1: удалось решить главную задачу проекта и адресовать его цель - см. Общий вывод ниже.

#### Вывод 2: данные можно сделать более содержательными для анализа, адресовав следующие замечания:
* желательно иметь на порядок больше данных для более достоверных выводов (все зависимости наблюдаются на грани пары дисперсий стат-погрешности)
* данные столбца `days_employed` (стаж работы) не удалось разумно истолковать - предполагается проблема конверсии типов и форматов при переносах данных, нужен повторный запрос и контроль сохранности информации по ним
* без идентификатора клиента или регистрационного номера процесса обработки заявки на кредит невозможно содержательно работать с дупликатами.  Оставшихся категорий банально не хватает для избежания случайных "тезок"-записей
* слишком много записей с отсутствующими данными - 10.1%, что и привело к проблеме возникновения фальшивых дупликатов
* категоризация типа занятости (`income_type`) в исходных данных не понятна - особенно по категориям 'сотрудник' и 'компаньон', а также 'предприниматель' (почему последних только 2 во всей статистике?).  Нужны разъяснения по значению данных категорий

#### Вывод 3: осталось неисследованным загадочное полное совпадение данных по стажу работы и ежемесячному доходу
возможно расследование причин может пролить свет на формат информации по стажу и восстановить ее

## Общий вывод:

Можно рекомендовать следующий скоринг кредитоспособности клиента для кредитного отдела:
* женатые и бездетные = +5 условных пунктов
* гражданский брак с одним ребенком = -3 условных пункта
* не женатые без детей = -2 условных пункта
* женатые с 2 детьми = -2 условных пункта
* вдовец без детей = -1 условный пункт
* не женатый с одним ребенком = -1 условный пункт
* остальные варианты = 0 условных пунктов

нормировка определяется весами других координат скоринга кредитоспособности - примерно один пункт соответствует росту вероятности отдачи на 1%.  На текущий момент общая вероятность возврата кредита в срок (21301-1724)/21301 = 91,9 %